In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
plt.style.use('seaborn-whitegrid')
plt.rc('font', size=11)
plt.rc('legend', fontsize=11)
plt.rc('lines', linewidth=2)
plt.rc('axes', linewidth=2)
plt.rc('axes', edgecolor='k')
plt.rc('xtick.major', width=2)
plt.rc('xtick.major', size=6)
plt.rc('ytick.major', width=2)
plt.rc('ytick.major', size=6)
plt.rc('pdf', fonttype=42)
plt.rc('ps', fonttype=42)

# absolute path to ST modules
module_path = r'C:\Users\Jose Alvarez\Documents\Projects\CounterfactualSituationTesting\src'
# module_path = os.path.abspath(os.path.join('../src')) # or the path to your source code
sys.path.insert(0, module_path)
# local files
from src.situation_testing.situation_testing import SituationTesting

In [2]:
# change data path accordingly
data_path = os.getcwd() + '\\' + 'data' + '\\'
resu_path = os.getcwd() + '\\' + 'results' + '\\'

In [3]:
# factual dataset
org_df    = pd.read_csv(data_path + 'Karimi2020_v2.csv', sep='|', )
# counterfactual dataset
org_cf_df = pd.read_csv(resu_path + 'counterfactuals\\cf_Karimi2020_v2.csv', sep='|', )

In [4]:
org_df.head(5)

,LoanApproval,AnnualSalary,AccountBalance,u1,u2,Gender
0,-1.0,35000,7947.678090,50000,-973.152642,1
1,1.0,120000,36940.097383,120000,940.097383,0
2,-1.0,90000,23564.129008,90000,-3435.870992,0
3,-1.0,80000,27596.570524,80000,3596.570524,0
4,1.0,201000,59008.567839,210000,-705.778380,1


In [5]:
org_cf_df.head(5)

,u_AnnualSalary,u_AccountBalance,AnnualSalary,AccountBalance,LoanApproval,Gender
0,-49858.94,-1452.13,50796.35,13852.05,-1.0,1
1,19344.71,950.86,120000.00,36940.10,1.0,0
2,-10655.29,-3458.07,90000.00,23564.13,-1.0,0
3,-20655.29,3563.38,80000.00,27596.57,-1.0,0
4,116141.06,-8.85,216796.35,64912.94,1.0,1


### Set overall paremeters for test
We can run standard ST, counterfactual ST with/without the search centers, and counterfactual fairness. Notice that CF is included within cfST when incuding the search centers: these are th factual and counterfactual instances.

In [6]:
# attribute-specific params
feat_trgt = 'LoanApproval'
feat_trgt_vals = {'positive': 1, 'negative': -1}
# list of relevant features
feat_rlvt = ['AnnualSalary', 'AccountBalance']
# protected feature
feat_prot = 'Gender'
# values for the protected feature: use 'non_protected' and 'protected' accordingly
feat_prot_vals = {'non_protected': 0, 'protected': 1}

# st-specific params
# size of neiuborhoods
n = 15
# significance level
alpha = 0.05
# tau diviation
tau = 0.0

In [7]:
res_df = org_df[['Gender', 'LoanApproval']].copy()
res_df['cf_LoanApproval'] = org_cf_df['LoanApproval'].copy()
res_df.head(5)

,Gender,LoanApproval,cf_LoanApproval
0,1,-1.0,-1.0
1,0,1.0,1.0
2,0,-1.0,-1.0
3,0,-1.0,-1.0
4,1,1.0,1.0


From one of the CLeaR reviewers: what is your Oracle?
I guess, in this world, it would be a world freed from systematic bias, meaning counterfactual fairness captures this discrepancy between worlds, no?

The world where $X_1$ and $X_2$ are born without $A$'s influence is a counterfactually fair world by definition from Kusner et al.!!!

In [18]:
print(res_df[(res_df['Gender'] == 1) & (res_df['LoanApproval'] == 1)].shape[0])
# vs
print(res_df[(res_df['Gender'] == 1) & (res_df['cf_LoanApproval'] == 1)].shape[0]) #ground truthc (if you will)

674
1050


In [23]:
test_df = org_df.copy()

st = SituationTesting()
st.setup_baseline(test_df, nominal_atts=['Gender'], continuous_atts=['AnnualSalary', 'AccountBalance'])

res_df['ST'] = st.run(target_att='LoanApproval', target_val={'positive': 1, 'negative': -1},
                      sensitive_att='Gender', sensitive_val={'non_protected': 0, 'protected': 1},
                      k=n, alpha=alpha, tau=tau)

standardizing factual dataset


In [24]:
# get all neighborhoods for ST
stST_knn = st.res_dict_df_neighbors

In [25]:
test_df    = org_df.copy()
test_cf_df = org_cf_df.copy()

# don't include the centers
cf_st = SituationTesting()
cf_st.setup_baseline(test_df, test_cf_df, nominal_atts=['Gender'], continuous_atts=['AnnualSalary', 'AccountBalance'])

res_df['cfST'] = cf_st.run(target_att='LoanApproval', target_val={'positive': 1, 'negative': -1},
                           sensitive_att='Gender', sensitive_val={'non_protected': 0, 'protected': 1},
                           include_centers=False,
                           k=n, alpha=alpha, tau=tau)

standardizing factual dataset
standardizing counterfactual dataset


In [26]:
# get all neighnorhoods for cfST
cfST_knn = st.res_dict_df_neighbors

stST vs cfST:
- look at the cfST without centers (to be able to compare them)
- look at the c's that are shared between both methods: look at the distribution of each's ctr and tst
    - also look at the factual vs counterfactuals: average distance among this set
- look at the c's that are NOT shared between both methods and do the same

In [44]:
# let's look at those c's that appear both in stST and cfST
test_c = res_df[(res_df['ST'] > tau) & (res_df['cfST'] > tau)].index.to_list()

for c in test_c:
    print(c)
    

5
147
181
264
420
502
798
926
1044
1306
1313
1364
1399
1536
1573
1607
1808
1941
1958
2048
2120
2285
2327
2446
2565
2751
2926
3065
3134
3223
3239
3267
3653
3688
3747
3797
3816
4059
4072
4125
4211
4237
4268
4391
4541
4571
4613
4631
4714
4749
4790
4862
4880
4907
4978


In [42]:
stST_knn[5]

{'ctr_idx': [751,
  4517,
  3798,
  4072,
  3223,
  2118,
  2327,
  4211,
  2926,
  4750,
  4447,
  2211,
  4391,
  1364,
  799],
 'tst_idx': [165,
  2608,
  2721,
  4252,
  155,
  536,
  104,
  1400,
  2921,
  848,
  2195,
  695,
  647,
  4736,
  2081]}

In [43]:
cfST_knn[5]

{'ctr_idx': [751,
  4517,
  3798,
  4072,
  3223,
  2118,
  2327,
  4211,
  2926,
  4750,
  4447,
  2211,
  4391,
  1364,
  799],
 'tst_idx': [165,
  2608,
  2721,
  4252,
  155,
  536,
  104,
  1400,
  2921,
  848,
  2195,
  695,
  647,
  4736,
  2081]}

In [ ]:
test_df    = org_df.copy()
test_cf_df = org_cf_df.copy()

# include the centers
cf_st = SituationTesting()
cf_st.setup_baseline(test_df, test_cf_df, nominal_atts=['Gender'], continuous_atts=['AnnualSalary', 'AccountBalance'])

res_df['cfSTwith'] = cf_st.run(target_att='LoanApproval', target_val={'positive': 1, 'negative': -1},
                               sensitive_att='Gender', sensitive_val={'non_protected': 0, 'protected': 1},
                               include_centers=True,
                               k=n, alpha=alpha, tau=tau)

In [ ]:
res_df['CF'] = cf_st.res_counterfactual_unfairness

In [ ]:
res_df[res_df['Gender']==1].head(10)

1) We need to compare ST vs cfST
- check that all STs are in cfST
- focus on ST $\cap$ cfST: plot the distribution of group vs test group per k for X1 and X2
- do the same for those not shared

We would like to see that when ST and cfST agree we get "tighter" control and test groups: notice that the control groups across tools here should be identical, meaning the SOURCE OF VARIATION COMES FROM THE TEST GROUP.

We can also play with/without center here, but since ST doesn't use the center, it makes sense to use cfSt without centers for this comparisson.

We want to show/make the case: idealized comparison vs fairness given the difference.

2) We also need to compare cfST vs CF
- check that all CFs are in cfST (at least when including the center)
- focus on cases where CF doesn't identify discrimination: highlight that we need more than the individual comparison

We would like to see the limitations of relying exclusively on counterfactual fairness for detecting discrimination: for the law it's not enough. Compare with and without centers: in the former, we are by default including CF in the comparisson and the variation around it. 

3) Finally, compare all three: when do they coincide?

In [ ]:
len(res_df[res_df['ST'] > tau].index)

In [ ]:
print(res_df[(res_df['ST'] > tau) & (res_df['cfST'] > tau)].shape)
print(res_df[(res_df['ST'] > tau) & (res_df['cfSTwith'] > tau)].shape) # all i in ST are in cfST: as expected

In [ ]:
len(res_df[res_df['CF'] == True].index)

In [ ]:
print(res_df[(res_df['cfST'] > tau) & (res_df['CF'] == True)].shape)
print(res_df[(res_df['cfSTwith'] > tau) & (res_df['CF'] == True)].shape) # all CFs are in cfST when including the centers: as expected
# then the CF is the lower bound (it seems) of cfST with centers!

In [ ]:
# explore the distributions... ctr group vs tst groups: show the disimilar individuals for the same i factual
# ... look at the cases where CF failed / didn't fail wrt to cfST: box plots???